# Imports

In [1]:
from pyspark.sql import DataFrame, functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, BooleanType, LongType, DoubleType, ArrayType, DateType, MapType
from pyspark.ml.feature import Bucketizer
from matplotlib import pyplot as plt
import seaborn as sns
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, MinHashLSH, MinHashLSHModel, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.stat import Summarizer
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import col
import pandas as pd
import numpy as np
from pyspark.sql import Window
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALSModel
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.evaluation import RankingMetrics


## Create Spark session

In [2]:
from dtu_ctfds_02807_proj.context_creation import create_context

local = True
sc, spark = create_context(local)

Cell has not been executed before, running...


24/12/06 14:37:11 WARN Utils: Your hostname, macbook.local resolves to a loopback address: 127.0.0.1; using 192.168.86.58 instead (on interface en0)
24/12/06 14:37:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/06 14:37:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Load data

In [3]:
if local:
    game_reviews = spark.read.parquet('data/many_reviews_users_parquet')
    games = spark.read.parquet("data/steam_games_parquet")
else:
    game_reviews = spark.read.parquet("file:////work/ds/steam_reviews_parquet")
    games = spark.read.parquet('file:////work/ds/steam_games_parquet')
    
game_reviews: DataFrame = game_reviews.persist()
games: DataFrame = games.persist()

In [4]:
game_reviews: DataFrame = game_reviews.persist()
games: DataFrame = games.persist()

# Content based filtering

In [5]:
# Select relevant columns for content-based filtering
game_data = games\
    .filter(games.tags.isNotNull())\
    .filter(games.detailed_description.isNotNull())\
    .select("app_id", "name", "genres", "publishers", "detailed_description", "tags")

# Preprocessing pipeline
# Tokenize the detailed description
tokenizer = Tokenizer(inputCol="detailed_description", outputCol="tokens")

# Remove stopwords
stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")

# Apply TF (Term Frequency)
hashing_tf = HashingTF(inputCol="filtered_tokens", outputCol="raw_features", numFeatures=1000)

# Apply IDF (Inverse Document Frequency)
idf = IDF(inputCol="raw_features", outputCol="text_features")

genres_vectorizer = CountVectorizer(inputCol="genres", outputCol="genres_vector", binary=True)
publishers_vectorizer = CountVectorizer(inputCol="publishers", outputCol="publishers_vector", binary=True)

# Convert tags (Map<String, Int>) into feature vector
# Extract keys and values from the tags map
tags_keys = F.map_keys(game_data.tags)
tags_values = F.map_values(game_data.tags)

# Create a DataFrame with tag names (keys) and their respective weights (values)
game_data = game_data\
    .withColumn("tags_keys", tags_keys)\
    .withColumn("tags_values", tags_values)

# Use CountVectorizer to convert the tag keys into a feature vector
tags_vectorizer = CountVectorizer(inputCol="tags_keys", outputCol="tags_key_features")

vector_assembler = VectorAssembler(
    inputCols=["text_features", "genres_vector", "publishers_vector", "tags_key_features"],
    outputCol="features"
)

# Locality Sensitive Hashing (LSH) for similarity search
minhash_lsh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=3)

# Build the pipeline
pipeline = Pipeline(stages=[
    tokenizer,
    stopwords_remover,
    hashing_tf,
    idf,
    genres_vectorizer,
    publishers_vectorizer,
    tags_vectorizer,
    vector_assembler,
    minhash_lsh
])

In [6]:
# Fit the pipeline to the data
content_based_model = pipeline.fit(game_data)
processed_games_df = content_based_model.transform(game_data)

In [7]:
pipeline.save("models/content_based_pipeline")

24/12/06 14:37:24 ERROR Instrumentation: java.io.IOException: Path models/content_based_pipeline already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at org.apache.spark.ml.Pipeline$PipelineWriter.super$save(Pipeline.scala:204)
	at org.apache.spark.ml.Pipeline$PipelineWriter.$anonfun$save$2(Pipeline.scala:204)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent(events.scala:174)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent$(events.scala:169)
	at org.apache.spark.ml.util.Instrumentation.withSaveInstanceEvent(Instrumentation.scala:42)
	at org.apache.spark.ml.Pipeline$PipelineWriter.$anonfun$save$1(Pipeline.scala:204)
	at org.apache.spark.ml.Pipeline$PipelineWriter.$anonfun$save$1$adapted(Pipeline.scala:204)
	at org.apache.spark.ml.util.In

Py4JJavaError: An error occurred while calling o709.save.
: java.io.IOException: Path models/content_based_pipeline already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at org.apache.spark.ml.Pipeline$PipelineWriter.super$save(Pipeline.scala:204)
	at org.apache.spark.ml.Pipeline$PipelineWriter.$anonfun$save$2(Pipeline.scala:204)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent(events.scala:174)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent$(events.scala:169)
	at org.apache.spark.ml.util.Instrumentation.withSaveInstanceEvent(Instrumentation.scala:42)
	at org.apache.spark.ml.Pipeline$PipelineWriter.$anonfun$save$1(Pipeline.scala:204)
	at org.apache.spark.ml.Pipeline$PipelineWriter.$anonfun$save$1$adapted(Pipeline.scala:204)
	at org.apac

# Finding similar games to a given game

In [8]:
# FIFA 23 = 1811260
# Counter Strike Zero = 80
# Garry's Mod = 4000

sample_game = processed_games_df.filter(games.app_id == 80).limit(1)
sample_game_features = sample_game.select("features").first()

print("Looking for games similar to:")
sample_game.show(truncate=False)

sample_game_features = DenseVector(sample_game_features["features"])

mh_model: MinHashLSHModel = content_based_model.stages[-1]

n_similar_games = 20
print(f"Finding top {n_similar_games} most similar games...")
mh_model.approxNearestNeighbors(
    dataset=processed_games_df, key=sample_game_features, numNearestNeighbors=20, distCol="JaccardDistance"
)\
    .orderBy(F.asc("JaccardDistance"))\
    .show(truncate=False)

Looking for games similar to:


+------+------------------------------+--------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------

+-------+-------------------------------------+--------------------------------------------------------------+---------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Finding similar games to a combination of games

In [9]:
# FIFA 23 = 1811260
# Counter Strike Zero = 80
# Garry's Mod = 4000

# Step 1: Filter the relevant games
sample_games = processed_games_df.filter(
    (processed_games_df.app_id == 80) |
    (processed_games_df.app_id == 1811260)
)

print("Looking for games similar to:")
sample_games.show(truncate=False)

# Step 2: Compute the mean feature vector using Summarizer
mean_features_row = sample_games.select(
    Summarizer.mean(col("features")).alias("mean_features")
).first()

# Extract mean features as a DenseVector
sample_game_features = DenseVector(mean_features_row["mean_features"])

# Step 3: Perform similarity search using MinHashLSH
mh_model: MinHashLSHModel = content_based_model.stages[-1]

n_similar_games = 20
print(f"Finding top {n_similar_games} most similar games...")
mh_model.approxNearestNeighbors(
    dataset=processed_games_df,
    key=sample_game_features,
    numNearestNeighbors=n_similar_games,
    distCol="JaccardDistance"
)\
    .orderBy(col("JaccardDistance").asc())\
    .show(truncate=False)


Looking for games similar to:


+-------+------------------------------+--------------------+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Finding top 20 most similar games...


+-------+------------------------------------------------------+--------------------------------------------------------------------+-----------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------